In [18]:
from psaw import PushshiftAPI
import datetime as dt
import pandas as pd
import time
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import nltk

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType, StringType
from textblob import TextBlob

In [19]:
reddit_titles_df = pd.read_parquet('./data/reddit_titles.parquet.gzip')

In [20]:
spark_session = SparkSession \
    .builder \
    .appName('Python Spark SQL basic example') \
    .getOrCreate()

In [21]:
df_sp = spark_session.createDataFrame(reddit_titles_df)

In [22]:
sentiment = udf(lambda x: TextBlob(x).sentiment[0])
spark_session.udf.register('sentiment', sentiment)
df_sp = df_sp.withColumn('compound', sentiment('title').cast('float'))

In [23]:
df = df_sp.toPandas()

In [24]:
df

,date,title,compound
0,2021-03-15 04:01:22,.1 bitcoin in 10 years,0.000000
1,2021-03-15 04:02:48,Question about mining,0.000000
2,2021-03-15 04:04:14,Finding This Tonight. Who has 2M and Time to L...,0.000000
3,2021-03-15 04:06:36,Is it unwise to invest 25% of my surgery savin...,0.000000
4,2021-03-15 04:07:16,A Variety of Popular Celebrities Are Selling N...,0.600000
...,...,...,...
7409,2021-04-14 03:44:45,Bitcoin Tsunami,0.000000
7410,2021-04-14 03:49:02,How is bitcoin8m created and why it won’t caus...,-0.120833
7411,2021-04-14 03:51:53,Bitcoin is not a currrency.,0.000000
7412,2021-04-14 03:56:00,Thoughts on Coinbase Stock as opposed to just ...,0.500000
